In [3]:
import os, sys
lib_path = os.path.abspath(os.path.join('../..','_Libraries'))
sys.path.append(lib_path)
from rlabs_libutils import DataStruct, select_data, create_outlier_df, find_nearest_above, uniquelist_withidx
from rlabs_liblinreg import * # new library for the linear regression functions
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from itertools import izip
#%matplotlib inline

#Read raw data
**Note**: a window will apear with which you can select the data, this window can be hidden by the browser

In [4]:
path = select_data()
ds = DataStruct(path)

Plaid-15.06.11_16.14_jl_nofix_replication_eyetracker_data.txt: eyetracker data (38 colums format)
Trial 1 - 1.5 % of data was lost. SNR: 0.928862926542. CV: 1.07658511436
Trial 2 - 0.7 % of data was lost. SNR: 0.407201998552. CV: 2.45578362473
Trial 3 - 1.3 % of data was lost. SNR: 0.761812301826. CV: 1.31265929626
Trial 4 - 2.2 % of data was lost. SNR: 0.823482607092. CV: 1.21435473122
Trial 5 - 3.1 % of data was lost. SNR: 0.806924052454. CV: 1.23927400225


**Create outlier DataFrame**

In [5]:
outlier_threshold = 100  # velocity values over 100 will be outliers.
ambiguousoutlier_th = 80 # velocity values between 80 and 100 will be ambiguous outliers.
filter_samples = 5       # the samples following an outlier will not be outliers.

df = create_outlier_df(ds,outlier_threshold = outlier_threshold, 
                      ambiguousoutlier_th = ambiguousoutlier_th, filter_samples = filter_samples)

C:\Python27\lib\site-packages\pandas\computation\expressions.py:21: UserWarning: The installed version of numexpr 1.4.2 is not supported in pandas and will be not be used
The minimum supported version is 2.1

  "version is 2.1\n".format(ver=ver), UserWarning)


#Compute linear regression of Gaze position between outliers

In [6]:
outlier_idx = np.where(df['Outlierfiltered'])[0]
n = len(outlier_idx)-1
lr_struct = []
for i in range(n):
    lr_struct.append(regressionbtwpoints(df, outlier_idx[i], outlier_idx[i+1]))

#Refine linear regression

**method 1:** Using ambiguous outliers

**method 2:** Spliting linear regression interval into segments

**To do**: weight the linear regression with the number of points used

# Use method 1

In [7]:
minintervallen = 50 # minimum interval length (samples)
minsamples = 10      # minimum number of samples for a segment of the interval
fs = 120.0           # sampling frequency of the tobii eyetracker
badfitth = 0.3       # bad fit threshold. a regresison fit with a r_squared value below this, will be refined
maxdivisions = 12    # maximum number of divisions to split the bad fits

outlier_idx = np.where(df['Outlierfiltered'])[0]
n = len(outlier_idx)-1

m1_struct = [None]*n

for i in range(n):
    if lr_struct[i]['r_squared'] < badfitth:

        # get bad fit number of samples
        nsamples = (df['time'][outlier_idx[i+1]] - df['time'][outlier_idx[i]])/1000.0 * fs
        
        if nsamples > minintervallen:
            itvl_start = outlier_idx[i]
            itvl_end = outlier_idx[i+1]
            
            # method 1: if there are ambiguous outliers within:
            amb_outlier_idx = np.where(df['isAmbiguousOutlier'])[0]
            rthere = np.sum((amb_outlier_idx >= itvl_start+minsamples)&(amb_outlier_idx <= itvl_end-minsamples))
            use_m1 = rthere > 0
            if use_m1:
                tmp_m1 = method1_useamboutls(df, itvl_start, itvl_end, minsamples)
                m1_struct[i] = tmp_m1

# Use method 2

In [8]:
minintervallen = 50 # minimum interval length (samples)
minsamples = 25      # minimum number of samples for a segment of the interval
fs = 120.0           # sampling frequency of the tobii eyetracker
badfitth = 0.3       # bad fit threshold. a regresison fit with a r_squared value below this, will be refined
maxdivisions = 12    # maximum number of divisions to split the bad fits

outlier_idx = np.where(df['Outlierfiltered'])[0]
n = len(outlier_idx)-1

rf_struct1 = [None]*n
rf_struct2 = [None]*n
rf_bfidx = [None]*n

for i in range(n):
    if lr_struct[i]['r_squared'] < badfitth:

        # get bad fit number of samples
        nsamples = (df['time'][outlier_idx[i+1]] - df['time'][outlier_idx[i]])/1000.0 * fs
        
        if nsamples > minintervallen:
            itvl_start = outlier_idx[i]
            itvl_end = outlier_idx[i+1]
            sgmts1, sgmts2, bfidx = method2_splitintrvl(df, itvl_start, itvl_end, maxdivisions, minsamples, fs = 120.0)
            
            rf_struct1[i] = sgmts1
            rf_struct2[i] = sgmts2
            rf_bfidx[i] = bfidx

# Get the best fit from the two methods

In [9]:
# badfit_idx = np.where(r_squared_array < badfitth)[0]
badfit_idx = np.where(np.array([fit['r_squared'] for fit in lr_struct]) < badfitth)[0]
refinedout = [None] * n

for badidx in badfit_idx:
    if m1_struct[badidx] != None:           
        cumulative_rsqrd_m1 = np.sum(np.power(np.array([item['r_value'] for item in m1_struct[badidx]]),2))
        cumulative_rsqrd_m2 = np.sum(np.power(np.array([rf_struct1[badidx][rf_bfidx[badidx]]['r_value'], rf_struct2[badidx][rf_bfidx[badidx]]['r_value']]),2))
        
        if cumulative_rsqrd_m1 < cumulative_rsqrd_m2:
            refinedout.insert(badidx, [rf_struct1[badidx][rf_bfidx[badidx]], rf_struct2[badidx][rf_bfidx[badidx]]])
        else:
            refinedout.insert(badidx, m1_struct[badidx])
            
        cumulative_rsqrd_rf = np.sum(np.power(np.array([item['r_value'] for item in refinedout[badidx]]),2))
                   
    elif rf_struct1[badidx] != None:
        refinedout.insert(badidx, [rf_struct1[badidx][rf_bfidx[badidx]], rf_struct2[badidx][rf_bfidx[badidx]]])

#CLASSIFY PERCEPTS

1 - Create new struct and fill it with good fits of the original linear regression and the best from the refined

In [10]:
newlrstruct = []                                  #
for i in range(n):                                # for all the outlier-intervals
    if i in badfit_idx and refinedout[i] != None: # if it is a bad fit in the original linear regression
        for item in refinedout[i]:                #
            newlrstruct.append(item)              #
    else:                                         # if it was a good fit
        newlrstruct.append(lr_struct[i])          #

2 - Define threshold of length of interval, slope and r squared

In [11]:
threslen = 30       # minimal segment length
thresslo = 0.0007   # slope threshold
thresrsq = badfitth # r squared threshold

3 - For each fit, compute its percept

In [13]:
for fit in newlrstruct:
    fit['percept'] = classifyfit(fit)          

In [12]:
def classifyfit(fit, threslen = 30, thresslo = 0.0007, thresrsq = 0.3):
    """
        Compute the percept for a given fit.
        Three possibilities: A, B, ambiguous
        
        Input:
        - fit: dict containing the result of a linear regression using regressionbtwpoints()
        - threslen: minimum interval length. If len(fit) < threslen, percept = ambiguous
        - thresslo: minimum slope absolute value.
        - thresrsq: minimum r squared value.
        
        Output:
        - percept: 'A', 'B', 'amg'
        
    """
    
    # get length of interval in samples:
    fit_len = fit['end_idx'] - fit['start_idx']

    # condition 1: interval larger than threslen:
    c1 = fit_len >= threslen

    # condition 2.1: absolute value of slope larger than thresslo:
    c21 = np.abs(fit['slope']) > thresslo

    # condition 2.2: slope value positive or negative:
    c22 = fit['slope'] > 0 # slope<0: A, slope>0: B

    # condition 3: r squared higher than thresrsq:
    c3 = fit['r_squared'] > thresrsq

    # classify percept -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --        
    if c1:                                      # is the interval greater than length threshold? YES:
        if c21:                                 # is the slope greater than slope threshold? YES:
            percept = 'B' if c22 else 'A'       # is the slope positive or negative?
        else:                                   # is the slope greater than slope threshold? NO:
            percept = 'ambg'                    # ambiguous percept
    
    else:                                       # is the interval greater than length threshold? NO:
        if c3:                                  # is the r squared greater than r squared threshold? YES:
            if c21:                             # is the slope greater than slope threshold? YES:
                percept = 'B' if c22 else 'A'   # is the slope positive or negative?
            else:                               # is the slope greater than slope threshold? NO:
                percept = 'ambg'                # ambiguous percept
        else:                                   # is the r squared greater than r squared threshold? NO
            percept = 'ambg'                    # ambiguous percept
            
    return percept

#PLOT

In [ ]:
# figure 1: a) gaze position with reported and extracted percepts
#			b) outliers vs no-outliers with reported and extracted percepts
a_color = 'gray'
b_color = 'lightgray'
c_color = 'tomato'

f, ax = plt.subplots(1, sharex = True)

ax.plot(df['time'], df['LEpos_int'], label = 'leftgazeX (interpolated)') 												# velocity
ax.set_title('Position trace (degrees)')

for event in ds.trial_ts:
    ax.plot((event, event), (np.min(df['LEpos_int']),np.max(df['LEpos_int'])), 'k-')								# line to indicate start and end of trial
for on, off in ds.A_ts:
    ax.axvspan(on, off, ymin=0.5, ymax=0.99, facecolor=a_color, linewidth=0, alpha=0.5, label = 'A percept') 			# reported percepts for a (button press)
for on, off in ds.B_ts:
    ax.axvspan(on, off, ymin=0.5, ymax=0.99, facecolor=b_color, linewidth=0, alpha=0.5, label = 'B percept') 			# reported percepts for b

for fit in newlrstruct:
    on  = fit['start']
    off = fit['end']
    
    if fit['percept'] == 'A':
        pltcolor = a_color
        label = 'A percept'
    elif fit['percept'] == 'B':
        pltcolor = b_color
        label = 'B percept'
    else:
        pltcolor = c_color
        pltlabel = 'ambg percept'
    
    ax.axvspan(on, off, ymin=0.01, ymax=.5, facecolor=pltcolor, linewidth=0, alpha=0.5, label = pltlabel)

ax.set_ylabel("Extracted percepts | Reported percepts")


# Get artists and labels for legend of first subplot
handles, labels = ax.get_legend_handles_labels()
labelsidx, labels = uniquelist_withidx(labels)
ax.legend([handles[idx] for idx in labelsidx], labels)
ax.set_xlabel('time (ms)')

f.suptitle(ds.filename.split()[0])
plt.show()